<a href="https://colab.research.google.com/github/gihoonpark/Psychological_orientation_prediction-LGBM-XGB-RandomForest/blob/main/%EC%8B%AC%EB%A6%AC%EC%84%B1%ED%96%A5_%EC%98%88%EC%B8%A1ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/데이터분석/open data/open data/train.csv')

In [ ]:
df

In [ ]:
df.info() # data type, nan 여부 확인

Preprocessing

In [ ]:
x = df.drop(['index', 'voted'], axis=1)
y = df.loc[:,'voted']
print(x)
x_dummy = pd.get_dummies(x)
print(x_dummy)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(x_dummy, y, test_size=0.2, random_state=42)
kfold = StratifiedKFold(n_splits=5, random_state=42)

Train

In [ ]:
# XGBoosting model
import xgboost as xgb

xgb = xgb.XGBClassifier()
xgb_param_grid = {'max_depth':[5,7],
                  'gamma':[1,10],
                  'learning_rate':[0.01]}
xgb_gridsearch = GridSearchCV(xgb, xgb_param_grid, scoring='accuracy', verbose=1, cv=kfold, n_jobs = -1)
xgb_gridsearch.fit(x_dummy, y)

xgb_best = xgb_gridsearch.best_estimator_ # best parameter를 낸 xgb 모형

In [ ]:
# Randomforest model
rfc = RandomForestClassifier()
rfc_param_grid = {'n_estimators':[200],
                  'max_depth':[5,7],
                  'bootstrap':[False],
                  'max_features':[5,7]}
rfc_gridsearch = GridSearchCV(rfc, rfc_param_grid, scoring='accuracy', verbose=1, cv=kfold)
rfc_gridsearch.fit(x_dummy, y)

rfc_best = rfc_gridsearch.best_estimator_

In [ ]:
# LGBM model
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
lgb.fit(x_dummy, y)

Test

In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/데이터분석/open data/open data/test_x.csv')
df_test.info()

In [ ]:
x_test = df_test.drop(['index'], axis=1)
print(x_test)
x_test_dummy = pd.get_dummies(x_test)
print(x_test_dummy)

Ensemble(Voting)

In [ ]:
votingC =VotingClassifier(estimators=[('xgb',xgb_best),('rfc', rfc_best),('lgb', lgb)], 
                          voting='hard')
votingC.fit(x_dummy, y)

In [ ]:
y_pred = votingC.predict(x_test_dummy)
print(y_pred)